<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/1_0_py_PHI_EXTRACTION_FROM_ZIP_RAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
!pip install sddk
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### authorize google sheets
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:
PHI_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1zfTw0Hf304maBmrYvaMxRLnv1zfAVFixrtGTTsLCcT4/edit?usp=sharing")

# Extract the data from a zipfile on sciencedata.dk


In [0]:
### to get the zip file into a response
response = conf[0].get(conf[1] + "SDAM_data/PHI/PHI_Petra.zip")

In [5]:
### check that the response is functional
response

<Response [200]>

In [6]:
%%time
zipped = zipfile.ZipFile(io.BytesIO(response.content))

CPU times: user 37.8 ms, sys: 4.37 ms, total: 42.2 ms
Wall time: 43.2 ms


In [7]:
# how many files we have?
namelist = zipped.namelist()[1:] ### index "0" is for main directory
len(namelist)

3363

In [8]:
namelist = [name for name in namelist if not "._" in name]
namelist = [name for name in namelist if  ".csv" in name]
namelist = [name for name in namelist if  "output_raw" in name]
len(namelist)

742

# Test with one file

In [0]:
test = pd.read_csv(io.BytesIO(zipped.read("PHI/output_raw/CEG.csv")), sep="@")

In [11]:
test.head()

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data
0,/text/328151?location=1&patt=&bookid=303&offse...,CEG,I 109,Regions,CEG\nI 109,Boiotia — Tanagra — ca. 600 BC — IG VII 579 — ...,Funerary epigram for Dermys and Kitylos. Lime...,5,"i, left figure.1\n\nii, right figure.2\n\niii,...",Δέρμυς.\n\nΚιτύλος.\n\nἈμφάλκες ἔστασ’ ἐπὶ Κιτ...
1,/text/328152?location=1&patt=&bookid=303&offse...,CEG,I 110,Regions,CEG\nI 110,Boiotia — Haliartos (Kastri Maziou) — ca. 500 ...,"Funerary epigram for Kallias, son of Aigisthos...",4,1\n\n\n,Καλλία\nΑἰγίθοιο {²⁶Αἰγίσθοιο}²⁶·\nτὺ δ’ εὖ πρ...
2,/text/328153?location=1&patt=&bookid=303&offse...,CEG,I 111,Regions,CEG\nI 111,Boiotia — Tanagra — ca. 500 BC? — IG VII 3501 ...,Funerary epigram for Gathon. Limestone tablet...,1,1,[π]αῖδες̣ ἐ̣[ποίεσ]α̣ν μ̣νᾶ̣μ̣’ ἐνθάδ̣[ε] πατρ...
3,/text/328154?location=1&patt=&bookid=303&offse...,CEG,I 112,Regions,CEG\nI 112,Boiotia — Thisbe — ca. 500 BC? — IG VII 2247 —...,Funerary epigram for Phanes. Funerary stele. ...,2,1\n,ἀσστοῖ[ς] καὶ χσένοισι Φάνες φίλος [ἐνθάδε κεῖ...
4,/text/328155?location=1&patt=&bookid=303&offse...,CEG,I 113,Regions,CEG\nI 113,Boiotia — Thespiai — Palaiopanagia — ca. 500-4...,"Funerary epigram for Oligeides, son of Osthilo...",6,1\n\n\n\n5\n,μνᾶμ’ ἐπ’ Ὀλιγέ-\nδαι {ι} μ’ ὁ πατὲρ ἐ-\nπέθεκ...


# Extracting the whole dataset

In [0]:
dfs_list = []
for filename in namelist:
  try:
    df = pd.read_csv(io.BytesIO(zipped.read(filename)))
    if len(df.columns)<9:
      df = pd.read_csv(io.BytesIO(zipped.read(filename)), sep="@")
    if len(df.columns)<9:
      print(filename)
  except:
    df = pd.read_csv(io.BytesIO(zipped.read(filename)), sep="@")
  df["filename"] = filename.rpartition("/")[2]
  df["PHI_ID"] = df.apply(lambda row: int(row["URL"].partition("text/")[2].partition("?")[0]), axis=1)
  dfs_list.append(df)

In [75]:
PHI_raw = pd.concat(dfs_list)
PHI_raw.shape

(214995, 12)

In [76]:
PHI_raw.reset_index(inplace=True)
PHI_raw.drop("index", axis=1, inplace=True)
PHI_raw

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,/text/237443?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,3",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,3",Att. — Athens: Akropolis — stoich. 28 — 303/2 BC,NaN,4,1\n\n\n,ἐπὶ Λε[ωστράτου ἄρχοντος ἐπὶ τῆς Κ]-\nεκρο[πίδ...,CSCA.csv,237443
1,/text/237444?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,4",Att. — Athens: EM — stoich. 35 — 306-302 BC,NaN,15,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15,[․]ομε̣[․․․․․․․․․․․․․28․․․․․․․․․․․․․ τῆς]\nπρυ...,CSCA.csv,237444
2,/text/237445?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 173,5",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 173,5",Att. — Athens — non-stoich.,NaN,16,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n,— — — — —υτω․․․․11․․․․․\n— — — — —ωντωι̣․․․․9․...,CSCA.csv,237445
3,/text/237446?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 9,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 9,4",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,NaN,2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΔ ․,CSCA.csv,237446
4,/text/237447?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 11,16",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 11,16",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,NaN,2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΓ Ε,CSCA.csv,237447
...,...,...,...,...,...,...,...,...,...,...,...,...
214990,/text/280102?location=1090&patt=&bookid=661&of...,IK Prusias ad Hypium,164,Regions\nAsia Minor\nBithynia,IK Prusias ad Hypium\n164,Bith. — Prusias ad Hypium (Üskübü),NaN,2,1\n,[— — —]#⁷ΣΙ[— — —]\n[— — —]#⁷ τοῦ Ἀ̣[— — —],IK-Prusias-ad-Hypium.csv,280102
214991,/text/280103?location=1090&patt=&bookid=661&of...,IK Prusias ad Hypium,165,Regions\nAsia Minor\nBithynia,IK Prusias ad Hypium\n165,Bith. — Prusias ad Hypium (Üskübü),NaN,1,1,[— — — ]ΟΣΚΗ[— — —],IK-Prusias-ad-Hypium.csv,280103
214992,/text/280104?location=1090&patt=&bookid=661&of...,IK Prusias ad Hypium,166,Regions\nAsia Minor\nBithynia,IK Prusias ad Hypium\n166,Bith. — Prusias ad Hypium (Üskübü),NaN,1,1,[— — —]ΗΡΑ[— — —],IK-Prusias-ad-Hypium.csv,280104
214993,/text/280105?location=1090&patt=&bookid=661&of...,IK Prusias ad Hypium,167,Regions\nAsia Minor\nBithynia,IK Prusias ad Hypium\n167,Bith. — Prusias ad Hypium (Üskübü),NaN,1,1,[— — —]ΛΕΡ[— — —],IK-Prusias-ad-Hypium.csv,280105


In [77]:
sddk.write_file("SDAM_data/PHI/PHI_raw.json", PHI_raw, conf)

Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_raw.json"


In [0]:
PHI_raw_by_500 = PHI_raw[PHI_raw["PHI_ID"].isin(range(1, 300000, 500))]

In [70]:
PHI_raw_by_500

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
315,/text/262001?location=1040&patt=&bookid=517&of...,Panamara,316,Regions\nAsia Minor\nCaria,Panamara\n316,IStr 427,NaN,2,1\n,"Διῒ Πανημέρῳ, κόμαι Εὐτύχεως· ἐπὶ ἱερέω[ς] Κλα...",Panamara.csv,262001
185,/text/266001?location=1673&patt=&bookid=587&of...,St.Pont. III,110a,Regions\nAsia Minor\nPontus and Paphlagonia,St.Pont. III\n110a,Pont. — Amasia — Rom. Imp. period,NaN,6,1\n\n\n\n5\n,Αὐρ(ηλίῳ) Φιλο-\nμούσῳ ❦\nἀρχιάτρῳ\nῬω[μᾶνα(?)...,St.Pont.-III.csv,266001
130,/text/231001?location=1365&patt=&bookid=394&of...,Agora XV,130,Regions\nAttica (IG I-III)\nAttica,Agora XV\n130,Att. — Athens: Agora — 220/19,NaN,149,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20...,ἐπὶ Μενεκράτου ἄρχοντος ἐπὶ τῆς Οἰνεῖδος ἕκτη-...,Agora-XV.csv,231001
317,/text/79501?location=126&patt=&bookid=23&offse...,"IG XII,8",311,"Regions\nAegean Islands, incl. Crete (IG XI-[X...","IG XII,8\n311",Thasos,NaN,7,1\n\n\n\n5\n\n,Ἐπικράτης Κτησιφῶντος\nΠυθίων Περιθύμου\nΠίν...,IG-XII-8.csv,79501
164,/text/218501?location=1497&patt=&bookid=367&of...,Koptos à Kosseir,163,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...",Koptos à Kosseir\n163,Eg. — el-Boueib,NaN,2,1\n,Κάλχων̣\nΠανὶ Χρ(υσόδωτηι).,Koptos---Kosseir.csv,218501
...,...,...,...,...,...,...,...,...,...,...,...,...
60,/text/220001?location=1497&patt=&bookid=376&of...,Prose sur pierre,60,"Regions\nEgypt, Nubia and Cyrenaïca\nEgypt and...",Prose sur pierre\n60,Eg. — Alexandria (El Iskandariya) — 89-91 AD —...,NaN,23,I.1\nII.2\n\n\n5\n\nIII.7\n\n\n10\n\n\nIV.13\n...,θεοῖς καταχθονίοις.\nΠο[μπ]ηία Μοῦσα ἀνειέρωσε...,Prose-sur-pierre.csv,220001
7,/text/189501?location=473&patt=&bookid=252&off...,IRT,264,Regions\nNorth Africa\nTripolitania,IRT\n264,Tripolitania — Lepcis Magna (Lebda) — 3rd c. A...,NaN,13,"a,pediment.1\n\nb,pilasters.1\n\n\n\n5\n\nc,ba...",ἀγαθῇ τύχῃ [τῶν κυ]ρίων\nἈσσκληπιάδης {²⁶Ἀσκλη...,IRT.csv,189501
322,/text/299001?location=1237&patt=&bookid=736&of...,"IDR III,2",310,Regions\nThrace and the Lower Danube (IG X)\nD...,"IDR III,2\n310",Dacia Sup. — Ulpia Traiana-Sarmizegetusa — 2nd...,NaN,3,1\n\n,[— Au]rel(ius) S[— — — pro sal(ute) sua]\n[et(...,IDR-III-2.csv,299001
180,/text/279001?location=1090&patt=&bookid=659&of...,"TAM IV,1",181,Regions\nAsia Minor\nBithynia,"TAM IV,1\n181",Bith. — Nikomedeia (Izmit),NaN,3,1\n\n,— — — Ἀράβιος ὁ θεῖος μου κὲ Μ— — —\n— — — Θεα...,TAM-IV-1.csv,279001


In [0]:
set_with_dataframe(PHI_overview.add_worksheet("PHI_raw_by_500", 1, 1), PHI_raw_by_500)

# Exporting list of inscriptions


In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_raw.json", "df", conf)

In [0]:
inscriptions_list = [int(url.partition("text/")[2].partition("?")[0]) for url in  PHI["URL"]]
print(inscriptions_list[:100])

[237766, 237767, 237768, 237769, 237770, 237771, 237772, 237773, 237774, 237775, 237776, 237777, 237778, 237779, 237780, 237781, 237782, 237783, 237784, 237785, 237786, 237787, 237788, 237789, 237790, 237791, 237792, 237793, 237794, 237795, 237796, 237797, 237798, 237799, 237800, 237801, 237802, 237803, 237804, 237805, 237806, 237807, 237808, 237809, 237810, 237811, 237812, 190454, 190455, 190457, 190458, 190459, 190460, 190461, 190462, 190463, 190464, 190465, 190466, 190467, 190468, 190469, 190470, 190471, 190472, 190473, 190474, 190475, 190476, 190477, 190478, 190479, 190480, 190481, 190482, 190483, 190484, 190485, 190486, 190487, 190488, 190489, 190490, 190491, 190492, 190493, 190494, 190495, 190496, 190497, 190498, 190499, 190500, 190501, 190502, 186278, 190503, 190504, 190505, 190506]


In [0]:
PHI["inscription ID"] = inscriptions_list

In [0]:
PHI.sort_values("inscription ID").head(10)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4,inscription ID
107154,/text/1754?location=1365&patt=&bookid=3&bookid...,IG I²,165,Regions\nAttica (IG I-III)\nAttica,IG I² 165,None,None,12.0,1\n\n\n\n5\n\n\n\n\n10\n\n,․․5․․οτο— — —\n— — —\n— — —\n․․․ρτ— — —\n․․ν ℎ...,Att.,None,None,Ath.: Od.Evangelistrias?,c. 2nd ac,c. 2nd ac,"IG I³, p.972",None,None,None,1754
107155,/text/1755?location=1365&patt=&bookid=3&bookid...,IG I²,185a,Regions\nAttica (IG I-III)\nAttica,IG I² 185a,None,None,5.0,1\n\n\n\n5,— — —․․5․․ο ℎεκαστ․․․— — —\n— — —․․․․λακες μὲ ...,Att.,None,None,prov.?,None,forgery?,"IG I³, p.972",None,None,None,1755
107156,/text/1756?location=1365&patt=&bookid=3&bookid...,IG I²,"400,Ib",Regions\nAttica (IG I-III)\nAttica,"IG I² 400,Ib",None,None,3.0,1\n\n,ℎοι ℎιππῆς ἀπὸ το͂ν πολεμίον ℎιππαρχ[όν]-\n[το...,Att.,None,None,Ath.: Akr.,None,early imp.,DAA 135a,None,None,None,1756
107157,/text/1757?location=1365&patt=&bookid=3&bookid...,IG I²,"400,II",Regions\nAttica (IG I-III)\nAttica,"IG I² 400,II",None,None,3.0,1\n\n,[ℎοι ℎι]π̣π̣ῆ̣[ς ⋮ ἀπὸ] το͂ν ⋮ π[ολ]ε[μί]ον ⋮ ...,Att.,None,None,Ath.: Akr.,None,early imp.,DAA 135b,None,None,None,1757
107158,/text/1758?location=1365&patt=&bookid=3&bookid...,IG I²,"503,adn",Regions\nAttica (IG I-III)\nAttica,"IG I² 503,adn",None,None,2.0,1\n,[–⏕—]δες ἀ[νέθεκε]\n[Διὸς κ]ρατερό[φρονι παιδί].,Att.,None,None,Ath.: Akr.,c. 500-490? BC,[pottery],c. 500-490? BC,Vasen.Akr. I.2570,None,None,1758
107159,/text/1759?location=1365&patt=&bookid=3&bookid...,IG I²,522,Regions\nAttica (IG I-III)\nAttica,IG I² 522,None,None,2.0,\n1,— — —\nἄνδρες ἐποίεσαν σοφίαισιν καλὸν ἄγαλμα.,Att.,None,None,Ath.: Akr.,None,[pottery],end 6th BC,Vasen.Akr. II.1348,None,None,1759
107160,/text/1760?location=1365&patt=&bookid=3&bookid...,IG I²,561,Regions\nAttica (IG I-III)\nAttica,IG I² 561,None,None,4.0,\n1\n\n,— — —\nἈχαρνέος θυγάτη̣[ρ]\nἈθηναίαι Ἐργάνηι\n...,Att.,stoich. 15,None,Ath.: Akr.,4th c. BC,4th c. BC,"IG I³, p. 972",None,None,None,1760
107161,/text/1761?location=1365&patt=&bookid=3&bookid...,IG I²,575,Regions\nAttica (IG I-III)\nAttica,IG I² 575,None,None,3.0,1\n\n,Διφιλίδες ἐκ Κερ̣αμέο[ν]\nἀνέθεκε.\nsigla: ΦΥΠ...,Att.,None,None,Ath.: Akr.,aet. Hadriani,aet. Hadriani,"IG I³, p. 972",None,None,None,1761
107162,/text/1762?location=1365&patt=&bookid=3&bookid...,IG I²,644,Regions\nAttica (IG I-III)\nAttica,IG I² 644,None,None,1.0,1,Σμῖκρος ἀνέθεκεν τε͂ι Ἀθενάαι.,Att.,None,None,Ath.: Akr.,None,[pottery],Vasen.Akr. II.238,None,None,None,1762
107163,/text/1763?location=1365&patt=&bookid=3&bookid...,IG I²,730,Regions\nAttica (IG I-III)\nAttica,IG I² 730,None,None,1.0,1,[— — ἀνέθεκ]εν #⁵⁶ ἀπαρχὲ<ν> #⁵⁶ ἔργον Ἀθεναία...,Att.,None,None,Ath.: Akr.,None,[pottery],Vasen.Akr. II.1354,None,None,None,1763


In [0]:
inscriptions_list_sorted = sorted([int(ins) for ins in inscriptions_list])
print(inscriptions_list_sorted[:1000])

[1754, 1755, 1756, 1757, 1758, 1759, 1760, 1761, 1762, 1763, 1764, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1786, 1787, 1788, 1789, 1790, 1791, 1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1825, 1826, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2241, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2252, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2260, 2261, 2262, 2263, 2264, 2265, 2266, 2267, 2268, 2269, 2270, 2271, 2272, 2273, 2274, 2275, 2276, 2277, 2278, 2279, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296, 2297, 2298, 2299, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2308, 2309, 2310, 2311, 231

In [0]:
sddk.write_file("SDAM_data/PHI/inscriptions_list_sorted.json", inscriptions_list_sorted, conf)

Your <class 'list'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/inscriptions_list_sorted.json"


In [0]:
### there was an update in the sddk package, which now manages encoding on its own
sddk.write_file("SDAM_data/PHI/PHI_sample_utf8.json", PHI[:10000], conf)

A file with the same name ("PHI_sample_utf8.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_sample_utf8.json"


In [0]:
### and now do the same with the whole dataset
sddk.write_file("SDAM_data/PHI/PHI_merged_utf8.json", PHI, conf)

A file with the same name ("PHI_merged_utf8.json") already exists in this location.
Press Enter to overwrite it or choose different path and filename: 
Your <class 'pandas.core.frame.DataFrame'> object has been succefully written as "https://sciencedata.dk/files/SDAM_root/SDAM_data/PHI/PHI_merged_utf8.json"


# Testing: reading sample digested by R by to Python

In [0]:
# reading the json processed in R back to Python:
PHI_sample = sddk.read_file("SDAM_data/PHI/PHI_tibble.json", "df", conf)
PHI_sample.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,region,stoich,sinistr,subregion,date,other_info_0,other_info_1,other_info_2,other_info_3,other_info_4
0,[/text/237766?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]",[10],[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 10]",{},{},[46],[5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n20\n\n\n\n...,[[․․․․․12․․․․․] ὀβολ[ὸν ․․5․․]\n[․․․․․12․․․․․]...,[Att.],[stoich. 23],{},[Ath.: Eleusinion],[ca. 460 BC],[ca. 460 BC],"[IG I² 6,C]","[I(3).6,C ll. 5-50]",{},{}
1,[/text/237767?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[18,H5]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 18,H5]",{},{},[35],[\n1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\...,[corona\nθ[εοί].\nΕ̣ὐθ̣[․․6․․․]θ̣ωνος Ἐλευσίνι...,[Att.],[stoich. 25],{},[Eleusis],[ca. mid-4th c. BC],[ca. mid-4th c. BC],[IG II² 1188],{},{},{}
2,[/text/237768?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[24,H11]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 24,H11]",{},{},[26],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n\n\n\n\n2...,[ἐπὶ Λυσιάδου ἄρχοντος Π̣[υανοψ]ιῶνος ἕκτ[ει ἐ...,[Att.],[non-stoich.],{},[Ath.: Agora],{},[148/7? BC],"[Hsp.11.1942.293,58 [SEG 19.124]]",{},{},{}
3,[/text/237769?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[30,H19]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 30,H19]",{},{},[5],[1\n\n\n\n5],[[Τ. Φλ]ά̣β. Πάντα[ινον]\n[Γαργ]ήττιον vac.\n[...,[Att.],{},{},[Athens],[ca. 100 AD?],[ca. 100 AD?],"[Hsp.35.1966.247,8 +]","[AD 25.1970.187,6 [SEG 23.114]]",[SEG 29.180],{}
4,[/text/237770?location=7&patt=&bookid=409&book...,"[Clinton, Sacred Officials]","[31,H20]",[Regions\nAttica (IG I-III)],"[Clinton, Sacred Officials 31,H20]",{},{},[14],[1\n\n\n\n5\n\n\n\n\n10\n\n\n\n],[[κ]α̣θ’ ὑπομνη[ματι]-\nσμὸν τῆς ἐξ Ἀ[ρείου]\n...,[Att.],{},{},[Eleusis],[166/7 AD],[166/7 AD],[IG II² 3984],{},{},{}
